In [32]:
import pandas as pd
import numpy as np

# pd.set_option("display.max_colwidth", 300)

df = pd.read_csv("results/outputs_scored.csv")
df.head()


,run_id,task_name,model,strategy,response,runtime_seconds,score,Unnamed: 7
0,54aad71e-1504-4b9f-a29d-e34da79f3bef,Code Generation,qwen2.5:1.5b,zero_shot,"```python\ndef categorize_numbers(numbers):\n even = []\n odd = []\n\n for number in numbers:\n if number % 2 == 0:\n even.append(number)\n else:\n odd.append(number)\n\n return {'even': even, 'odd': odd}\n\n# Example usage and check function\nnumb...",5.181,2.0,NaN
1,d6862ed8-d1a7-4bcf-928a-c7e4e61e17fd,Code Generation,qwen2.5:1.5b,few_shot,"```python\ndef create_even_odd_dict(numbers):\n even_numbers = [num for num in numbers if num % 2 == 0]\n odd_numbers = [num for num in numbers if num % 2 != 0]\n \n return {'even': even_numbers, 'odd': odd_numbers}\n```",2.135,2.0,NaN
2,1e3564fa-7466-4ac4-960d-0eb47de8f972,Code Generation,qwen2.5:1.5b,cot,"### Reasoning:\n\n1. We need to iterate through the input list of integers.\n2. For each integer, check if it is even or odd:\n - If divisible by 2 without a remainder (i.e., `num % 2 == 0`), it's even.\n - Otherwise, it's odd.\n3. Collect all even numbers into one list and all odd numbers i...",6.365,3.0,NaN
3,8f2649a4-d6c6-43f5-ab87-0c019cc01929,Code Generation,deepseek-r1:7b,zero_shot,"```python\ndef separate_even_odd(numbers):\n even = []\n odd = []\n for num in numbers:\n if num % 2 == 0:\n even.append(num)\n else:\n odd.append(num)\n return {'even': even, 'odd': odd}\n```",37.981,3.0,NaN
4,d7aae9d7-e563-47d1-9cae-018b7c99227c,Code Generation,deepseek-r1:7b,few_shot,"To solve the problem of creating a Python function that takes a list of integers and returns a dictionary with two keys: 'even' and 'odd', we can follow these steps:\n\n1. **Define the Function**: Create a function named `count_even_odd` that accepts a single parameter, which is the list of inte...",46.380,NaN,NaN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   run_id           50 non-null     object 
 1   task_name        45 non-null     object 
 2   model            50 non-null     object 
 3   strategy         50 non-null     object 
 4   response         46 non-null     object 
 5   runtime_seconds  50 non-null     float64
 6   score            45 non-null     float64
 7   Unnamed: 7       10 non-null     object 
dtypes: float64(2), object(6)
memory usage: 3.3+ KB


In [33]:
# fix the missing task name for Reading Comprehension
df["task_name"] = df["task_name"].fillna("Reading Comprehension")


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   run_id           50 non-null     object 
 1   task_name        50 non-null     object 
 2   model            50 non-null     object 
 3   strategy         50 non-null     object 
 4   response         46 non-null     object 
 5   runtime_seconds  50 non-null     float64
 6   score            45 non-null     float64
 7   Unnamed: 7       10 non-null     object 
dtypes: float64(2), object(6)
memory usage: 3.3+ KB


In [35]:
missing_scores = df[df["score"].isna()]
missing_scores[["task_name", "model", "strategy"]]

,task_name,model,strategy
4,Code Generation,deepseek-r1:7b,few_shot
13,Creative Writing,deepseek-r1:7b,zero_shot
18,Ethical Reasoning & Nuance,deepseek-r1:7b,zero_shot
19,Ethical Reasoning & Nuance,deepseek-r1:7b,few_shot
29,Instruction Following,deepseek-r1:7b,few_shot


In [36]:
df_valid = df.dropna(subset=["score"])

mean_by_model_strategy = (
    df_valid
    .groupby(["model", "strategy"])["score"]
    .mean()
    .reset_index()
)

mean_by_model_strategy

,model,strategy,score
0,deepseek-r1:7b,few_shot,2.428571
1,deepseek-r1:7b,zero_shot,2.500000
2,qwen2.5:1.5b,cot,2.000000
3,qwen2.5:1.5b,few_shot,2.400000
4,qwen2.5:1.5b,zero_shot,2.100000


In [37]:
mean_by_task_model = (
    df_valid
    .groupby(["task_name", "model"])["score"]
    .mean()
    .reset_index()
    .rename(columns={"score": "mean_score"})
)

mean_by_task_model


,task_name,model,mean_score
0,Code Generation,deepseek-r1:7b,3.000000
1,Code Generation,qwen2.5:1.5b,2.333333
2,Common Sense Reasoning,deepseek-r1:7b,2.500000
3,Common Sense Reasoning,qwen2.5:1.5b,1.666667
4,Creative Writing,deepseek-r1:7b,2.000000
5,Creative Writing,qwen2.5:1.5b,2.000000
6,Ethical Reasoning & Nuance,qwen2.5:1.5b,1.000000
7,Factual Knowledge & Retrieval,deepseek-r1:7b,3.000000
8,Factual Knowledge & Retrieval,qwen2.5:1.5b,3.000000
9,Instruction Following,deepseek-r1:7b,2.000000


In [38]:
overall_mean_by_model = (
    df
    .groupby("model")["score"]
    .mean()
    .reset_index()
    .rename(columns={"score": "mean_score"})
    .sort_values("mean_score", ascending=False)
)

overall_mean_by_model

,model,mean_score
0,deepseek-r1:7b,2.466667
1,qwen2.5:1.5b,2.166667
